# import package

In [1]:
from tabula import read_pdf
import pandas as pd
import datetime
import os

In [2]:
#設定pandas 的顯示上限
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 設定路徑，批次讀取pdf檔案

In [3]:
#0.get fileNames autimatically and set automation parameters
os.chdir('Rabies')
lst = os.listdir('106年')
lst


['desktop.ini',
 'done',
 'Rabies ppt 1060126.pdf',
 'Rabies ppt 1060224.pdf',
 'Rabies ppt 1060331.pdf',
 'Rabies ppt 1060428.pdf',
 'Rabies ppt 1060531.pdf',
 'Rabies ppt 1060630.pdf',
 'Rabies ppt 1060728.pdf',
 'Rabies ppt 1060831.pdf',
 'Rabies ppt 1060930.pdf',
 'Rabies ppt 1061031.pdf',
 'Rabies ppt 1061130.pdf',
 'Rabies ppt 1061229.pdf']

In [4]:
Pdf = []
for i in range(len(lst)):
    if '.pdf' in lst[i]:
        Pdf.append(lst[i])
    
Pdf

['Rabies ppt 1060126.pdf',
 'Rabies ppt 1060224.pdf',
 'Rabies ppt 1060331.pdf',
 'Rabies ppt 1060428.pdf',
 'Rabies ppt 1060531.pdf',
 'Rabies ppt 1060630.pdf',
 'Rabies ppt 1060728.pdf',
 'Rabies ppt 1060831.pdf',
 'Rabies ppt 1060930.pdf',
 'Rabies ppt 1061031.pdf',
 'Rabies ppt 1061130.pdf',
 'Rabies ppt 1061229.pdf']

In [5]:
fileName =[]
for i in range(len(Pdf)):
    fileName.append(Pdf[i][:-4])
fileName

['Rabies ppt 1060126',
 'Rabies ppt 1060224',
 'Rabies ppt 1060331',
 'Rabies ppt 1060428',
 'Rabies ppt 1060531',
 'Rabies ppt 1060630',
 'Rabies ppt 1060728',
 'Rabies ppt 1060831',
 'Rabies ppt 1060930',
 'Rabies ppt 1061031',
 'Rabies ppt 1061130',
 'Rabies ppt 1061229']

In [52]:
# 手動設定每個檔案的結尾頁
#can't automatically use...
endpage =[]
endpage.append('-7') #1
endpage.append('') #2
endpage.append('-8') #3
endpage.append('-8') #4
endpage.append('-9') #5
endpage.append('-9') #6
endpage.append('-10') #7
endpage.append('-10') #8
endpage.append('-8') #9
endpage.append('-8') #10
endpage.append('-9') #11
endpage.append('-7') #12

endpage

['-7', '', '-8', '-8', '-9', '-9', '-10', '-10', '-8', '-8', '-9', '-7']

# 以迴圈對指定檔案做爬蟲，將資料輸出為csv檔

In [53]:
Num= 0
Year = 106

while True:    
    #1. load the dataset from pdf
    df = read_pdf(str(Year)+'年/'+Pdf[Num],spreadsheet = True, pages = '6'+endpage[Num])
    #search the strange rows and get the indexes
    StrangeRows = df.index[df['序號'].isin(['序號'])].tolist()
    
    #2. 改掉奇怪的column names
    df=df.rename(columns = {'FAT\r檢測結果':'FAT檢測結果','確診日期註1':'確診日期'}) #change name of the column
    
    # 3. 刪除多餘的row
    df= df.drop(StrangeRows,axis = 0)
    
    #4. 重新排列df_new中的column
    #to replace the column, we have to get the list of all column names 
    cols = df.columns.tolist() 
    #Rearrange cols in any way you want
     #要記得用list的方法來處理他們
    lst = cols[2:]
    lst.append(cols[1])
    lst.insert(0,cols[0])
    df = df[lst]#rearrange the columns
    
    #5. change the date as digital format, and replace the year as 2018
    df['確診日期'] = pd.to_datetime(df['確診日期'],format = '%m月%d日')
    df['送檢日期'] = pd.to_datetime(df['送檢日期'],format = '%m月%d日')
    df_d1= df['確診日期']#get the date to rename the year
    df_d2= df['送檢日期']#get the date to rename the year
    #lambda dt: => means passing an anonymons function as an argumentto apply()
    new1 = df_d1.apply(lambda dt: dt.replace(year = Year+1911))
    new2 = df_d2.apply(lambda dt: dt.replace(year = Year+1911))
    #replace the original date format
    df['確診日期'] = new1
    df['送檢日期'] = new2
    
    #多的步驟:要把一月份中，年底的資料改成前一年
    if int(Pdf[Num][-8:-6]) == 1:
        Dec = pd.DatetimeIndex(df['送檢日期']).month == 12
        df_2017 = df[Dec]
        df_2017_d= df_2017['送檢日期']#get the date to rename the year
        new2017 = df_2017_d.apply(lambda dt: dt.replace(year = Year+1911-1))
        df.loc[Dec ,'送檢日期'] = new2017
    
    #6. create a new column for month, year, and day
    df['year'] = pd.DatetimeIndex(df['送檢日期']).year
    df['month'] = pd.DatetimeIndex(df['送檢日期']).month
    df['day'] = pd.DatetimeIndex(df['送檢日期']).day
    #輸出成csv檔案
    df.to_csv(str(Year)+'年/'+fileName[Num]+'.csv',index = False, encoding = 'utf_8_sig')
       
    del df
    Num +=1
    if Num > 11:
        break



# done~!